### Tabnet

In [1]:
# 패키지 불러오기
import pandas as pd
import numpy as np
import time
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import torch

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [2]:
# 데이터 불러오기
x_train = pd.read_csv('../data/fill_data/x_train.csv')
x_valid = pd.read_csv('../data/fill_data/x_valid.csv')
y_train = pd.read_csv('../data/fill_data/y_train.csv')
y_valid = pd.read_csv('../data/fill_data/y_valid.csv')

In [3]:
x_train = x_train.values
x_valid = x_valid.values
y_train = y_train.values
y_valid = y_valid.values

In [4]:
y_train = np.array(y_train).ravel()
y_valid = np.array(y_valid).ravel()

modeling

In [5]:
start_time = time.time()
clf = TabNetClassifier(
    n_d=56, 
    n_steps=3, gamma=1.4,
    lambda_sparse=1.1694994589239397e-05, 
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
    mask_type='sparsemax',
    n_shared=2,
    scheduler_params=dict(mode="min",
        patience = 3, # changing sheduler patience to be lower than early stopping patience
        min_lr=1e-5,factor=0.5,),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    verbose=0)
clf.fit(x_train, y_train,
    eval_set=[(x_train, y_train),(x_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    drop_last = False,
    patience = 29, max_epochs = 80
)
end_time = time.time()
print('걸린 시간 :', end_time - start_time)

## 기본값: 0.81477


Early stopping occurred at epoch 42 with best_epoch = 13 and best_valid_accuracy = 0.82365


/Users/yschoi/bitamin/ChurnProject/venv/lib/python3.7/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


걸린 시간 : 775.2704820632935


In [7]:
# prediction and evaluation
# train data
train_pred_y = clf.predict(x_train)
print('Accuracy: ', (accuracy_score(y_train, train_pred_y)))
print('f1_score: ', f1_score(y_train, train_pred_y))

# valid data
valid_pred_y = clf.predict(x_valid)
print('Accuracy: ', (accuracy_score(y_valid, valid_pred_y)))
print('f1_score: ', f1_score(y_valid, valid_pred_y))

Accuracy:  0.8240567988140021
f1_score:  0.9012676853423883
Accuracy:  0.8236545682102628
f1_score:  0.9008514530997115


하이퍼파라미터 튜닝

In [8]:
def Objective(trial : Trial, x_train, y_train, x_valid, y_valid):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
        lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
        mask_type=mask_type, n_shared=n_shared,
        scheduler_params=dict(mode="min",
        patience=trial.suggest_int("patienceScheduler",low=3,high=10), # changing sheduler patience to be lower than early stopping patience
        min_lr=1e-5,factor=0.5,),
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        verbose=0,
        ) #early stopping
    
    clf = TabNetClassifier(**tabnet_params)
    clf.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)],
        patience=trial.suggest_int("patience",low=15,high=30), max_epochs=trial.suggest_int('epochs', 1, 100),
        eval_metric=['accuracy'])

    score = f1_score(clf.predict(x_valid), y_valid)
    
    return score 


In [9]:
study = optuna.create_study(direction="maximize", sampler = TPESampler(), study_name='TabNet optimization')
study.optimize(lambda trial : Objective(trial, x_train, y_train, x_valid, y_valid), n_trials = 10)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2023-01-31 22:39:37,221] A new study created in memory with name: TabNet optimization



Early stopping occurred at epoch 35 with best_epoch = 17 and best_val_1_accuracy = 0.81615


/Users/yschoi/bitamin/ChurnProject/venv/lib/python3.7/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-01-31 23:02:16,646] Trial 0 finished with value: 0.8973516875131019 and parameters: {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 3, 'gamma': 1.0, 'n_shared': 3, 'lambda_sparse': 0.0008333446335986498, 'patienceScheduler': 5, 'patience': 18, 'epochs': 92}. Best is trial 0 with value: 0.8973516875131019.



Early stopping occurred at epoch 35 with best_epoch = 5 and best_val_1_accuracy = 0.82591


/Users/yschoi/bitamin/ChurnProject/venv/lib/python3.7/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-01-31 23:15:30,668] Trial 1 finished with value: 0.9021731486039806 and parameters: {'mask_type': 'sparsemax', 'n_da': 60, 'n_steps': 2, 'gamma': 1.2, 'n_shared': 2, 'lambda_sparse': 3.848386554022774e-05, 'patienceScheduler': 5, 'patience': 30, 'epochs': 43}. Best is trial 1 with value: 0.9021731486039806.


...


Early stopping occurred at epoch 20 with best_epoch = 3 and best_val_1_accuracy = 0.82178


/Users/yschoi/bitamin/ChurnProject/venv/lib/python3.7/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-02-01 00:35:04,554] Trial 8 finished with value: 0.9000561482313307 and parameters: {'mask_type': 'entmax', 'n_da': 64, 'n_steps': 2, 'gamma': 1.2, 'n_shared': 1, 'lambda_sparse': 5.5943576551246625e-06, 'patienceScheduler': 8, 'patience': 17, 'epochs': 37}. Best is trial 7 with value: 0.9024751427966998.



Early stopping occurred at epoch 22 with best_epoch = 3 and best_val_1_accuracy = 0.82203


/Users/yschoi/bitamin/ChurnProject/venv/lib/python3.7/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-02-01 00:42:41,579] Trial 9 finished with value: 0.8999577880962432 and parameters: {'mask_type': 'entmax', 'n_da': 60, 'n_steps': 1, 'gamma': 1.0, 'n_shared': 3, 'lambda_sparse': 3.6794820151366092e-06, 'patienceScheduler': 9, 'patience': 19, 'epochs': 93}. Best is trial 7 with value: 0.9024751427966998.


Best trial : score 0.9024751427966998, 
params {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 2, 'gamma': 1.4, 'n_shared': 3, 'lambda_sparse': 9.03897160309517e-05, 'patienceScheduler': 9, 'patience': 15, 'epochs': 13}
